## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    'Education', 'Age', 'DistanceFromHome',
    'JobSatisfaction', 'OverTime','StockOptionLevel',
    'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion',
    'NumCompaniesWorked'
]

# Create X_df using your selected columns
X_df = attrition_df[X_columns]


# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts().rename_axis('OverTime').to_frame('count')


,count
OverTime,
No,798
Yes,304


In [8]:
# Convert the OverTime column to numeric in X_train and X_test
X_train = pd.get_dummies(X_train, columns=['OverTime'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['OverTime'], drop_first=True)


In [9]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [10]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = encoder.transform(y_train[['Department']])
y_test_department = encoder.transform(y_test[['Department']])

y_train_department[:5]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [11]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = encoder.transform(y_train[['Attrition']])
y_test_attrition = encoder.transform(y_test[['Attrition']])

y_train_attrition[:5]

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Part 2: Create, Compile, and Train the Model

In [12]:
# Find the number of columns in the X training data.
input_dim = X_train.shape[1]

# Create the input layer
inputLayer = layers.Input(shape=(input_dim,), name='inputLayer')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(inputLayer)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)


In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)



In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name="attrition_hidden")(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_hidden)


In [15]:
# Create the model
model = Model(inputs=inputLayer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'categorical_crossentropy'
},
    metrics={
        'department_output': 'accuracy',
        'attrition_output': 'accuracy'
    }
)

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputLayer          │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ inputLayer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'department_output': y_train_department,
        'attrition_output': y_train_attrition
    },
    epochs=100,
    batch_size=32,
    validation_data=(X_test_scaled, {
        'department_output': y_test_department,
        'attrition_output': y_test_attrition
    })
)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - attrition_output_accuracy: 0.7311 - attrition_output_loss: 0.5596 - department_output_accuracy: 0.6152 - department_output_loss: 0.8918 - loss: 1.4508 - val_attrition_output_accuracy: 0.8152 - val_attrition_output_loss: 0.4787 - val_department_output_accuracy: 0.6467 - val_department_output_loss: 0.7868 - val_loss: 1.2846
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8359 - attrition_output_loss: 0.4134 - department_output_accuracy: 0.6772 - department_output_loss: 0.7707 - loss: 1.1842 - val_attrition_output_accuracy: 0.8207 - val_attrition_output_loss: 0.4378 - val_department_output_accuracy: 0.6413 - val_department_output_loss: 0.7785 - val_loss: 1.2369
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8644 - attrition_output_loss: 0.3538 - department_output_accuracy: 0.6499 - department_output_loss: 0.7790 - loss: 1.1332 - val_attrition_output_accuracy: 0.8315 - 

In [17]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {
    'department_output': y_test_department,
    'attrition_output': y_test_attrition
}, return_dict=True)
results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - attrition_output_accuracy: 0.7735 - attrition_output_loss: 1.9901 - department_output_accuracy: 0.5461 - department_output_loss: 3.1345 - loss: 5.1437


{'attrition_output_accuracy': 0.804347813129425,
 'attrition_output_loss': 1.708520770072937,
 'department_output_accuracy': 0.529891312122345,
 'department_output_loss': 3.2407875061035156,
 'loss': 5.073272228240967}

In [18]:
# Print the accuracy for both department and attrition
print(f"Attrition predictions accuracy: {results['attrition_output_accuracy']}")
print(f"Department predictions accuracy: {results['department_output_accuracy']}")

Attrition predictions accuracy: 0.804347813129425
Department predictions accuracy: 0.529891312122345


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, because if one class (like "No Attrition") is much more common, the model could just guess that every time and still look "accurate" — but it wouldn’t actually be helpful. Other metrics like precision can show how well the model handles both classes.

2. Use sigmoid for binary classification (yes or no), because it gives a result between 0 and 1 — like a probability.

3. - Use more data or balance the classes
   - Try different models or more layers
   - Tune hyperparameters (like learning rate or batch size)
   - Use regularization to prevent overfitting